Housing Prices Worksheet

Predicts Housing Prices Using Housing Prices.

In [63]:
import numpy as np
import pandas as pd

from scipy import stats
import statsmodels.api as sm 
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

Importing the Datasets

In [64]:
train_set = pd.read_csv(r"C:\Users\Malcolm Wright\Desktop\Housing Prices\Housing Price\train.csv")
test_set = pd.read_csv(r"C:\Users\Malcolm Wright\Desktop\Housing Prices\Housing Price\test.csv")


Display Training Datasets

In [65]:
train_set.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [66]:
sum(train_set["MSSubClass"].isna())

0

In [67]:

X = train_set.drop(['Id'], axis=1)

In [68]:
Y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [69]:
train, test, train_labels, test_labels = train_test_split(X,
                                         Y, 
                                         test_size = 0.3,)

Testing Regression

In [70]:
train_labels = np.array(train_labels)

Scenario One: All Variables

In [71]:
features = ['LotArea', 'OverallQual', 'BedroomAbvGr', 'TotRmsAbvGrd', 'GarageArea']

X1 = sm.add_constant(train[features])
reg = sm.OLS(train_labels, X1).fit()

reg.summary()

C:\Users\Malcolm Wright\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.719
Model:                            OLS   Adj. R-squared:                  0.718
Method:                 Least Squares   F-statistic:                     520.5
Date:                Tue, 31 Mar 2020   Prob (F-statistic):          3.10e-277
Time:                        14:29:16   Log-Likelihood:                -12320.
No. Observations:                1022   AIC:                         2.465e+04
Df Residuals:                    1016   BIC:                         2.468e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.123e+05   7177.949    -15.652      0.000   -1.26e+05   -9.83e+04
LotArea          0.9927      0.136      7.307      0.000       0.726       1.259
OverallQual   3.217e+04   1205.368     26.688      0.000    2.98e+04    3.45e+04
BedroomAbvGr -1.066e+04   2263.412     -4.711      0.000   -1.51e+04   -6222.562
TotRmsAbvGrd  1.331e+04   1246.502     10.677      0.000    1.09e+04    1.58e+04
GarageArea      63.1454      7.599      8.310      0.000      48.235      78.056
==============================================================================
Omnibus:                      349.129   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9306.005
Skew:                           0.967   Prob(JB):                         0.00
Kurtosis:                      17.656   Cond. No.                     8.08e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.08e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Scenario Two

In [72]:
X1 = sm.add_constant((train[['GarageArea']]))
reg = sm.OLS(train_labels, X1).fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.359
Model:                            OLS   Adj. R-squared:                  0.359
Method:                 Least Squares   F-statistic:                     572.2
Date:                Tue, 31 Mar 2020   Prob (F-statistic):          9.69e-101
Time:                        14:29:16   Log-Likelihood:                -12742.
No. Observations:                1022   AIC:                         2.549e+04
Df Residuals:                    1020   BIC:                         2.550e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       7.459e+04   4846.779     15.390      0.000    6.51e+04    8.41e+04
GarageArea   222.2017      9.289     23.921      0.000     203.974     240.430
==============================================================================
Omnibus:                      387.886   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3294.483
Skew:                           1.504   Prob(JB):                         0.00
Kurtosis:                      11.266   Cond. No.                     1.29e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.29e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Random Forests

In [73]:
iowa_model = DecisionTreeRegressor(random_state=1)
# Fit Model
iowa_model.fit(train[features], train_labels)

# Make validation predictions and calculate mean absolute error
val_predictions = iowa_model.predict(test[features])
val_mae = mean_absolute_error(val_predictions, test_labels)
print("Validation MAE when not specifying max_leaf_nodes: {:,.0f}".format(val_mae))

Validation MAE when not specifying max_leaf_nodes: 36,644


In [74]:
# Using best value for max_leaf_nodes
iowa_model = DecisionTreeRegressor(max_leaf_nodes=100, random_state=1)
iowa_model.fit(train[features], train_labels)

val_predictions = iowa_model.predict(test[features])
val_mae = mean_absolute_error(val_predictions, test_labels)
print("Validation MAE for best value of max_leaf_nodes: {:,.0f}".format(val_mae))

Validation MAE for best value of max_leaf_nodes: 30,514


Preparing Submissions

In [96]:
sub_data = test_set[features]
sub_data = sub_data.fillna(0)


In [97]:
sub_predictions = iowa_model.predict(sub_data)
sub_predictions

array([151030.22058824, 168925.        , 151030.22058824, ...,
       172290.        , 129062.5       , 222400.16666667])

In [100]:
sub_predictions = pd.DataFrame(sub_predictions, index = test_set["Id"], columns = ["SalePrice"])
sub_predictions

,SalePrice
Id,
1461,151030.220588
1462,168925.000000
1463,151030.220588
1464,175672.822581
1465,263000.000000
1466,175672.822581
1467,163044.697674
1468,175672.822581
1469,191666.581818


Export Submission

In [101]:
sub_predictions.to_csv(r"C:\Users\Malcolm Wright\Desktop\Housing Prices\Housing Price\Submission.csv")